# Datasets Related to Airports and Disaster Risk

The following sections are publically available datasets having to do with the world's airports. The source of the data, as well as a brief description and reflection are also provided. Ultimately, the data that are most relevant for humanitarian assessments for airport disaser preparedness programmes are combined into a single "master" spreadsheet. 

In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
pd.options.display.max_columns = 100

# <font color='black'> Table 1: OurAirports dataset </font>
**Source**: The file "data/airport-codes.csv" contains the list of all airports and their identifying codes. 

**Link**: CSV obtained from https://github.com/datasets/airport-codes. Original source url is http://ourairports.com/data/airports.csv where the information is compiled from multiple different sources and is updated nightly.

**Summary**: For purposes of this project, the ICAO code is the primary identifier used. The ICAO airport code is a four letter code used by Air Traffic Control systems around the world. Commercial applications may alternatively use the IATA code, which is a three-letter code  used in passenger reservation, ticketing and baggage-handling systems, but that is not used here since not all airports have an IATA code. 

In [6]:
df_OurAirports = pd.read_csv('data/airports.csv') # use the "ident" column for ICAO codes
df_OurAirports.dropna(subset=['ident'], inplace=True)   # drop airports without iata codes listed # None

# where "home_link" column is blank, fill it with "wikipedia_link"
df_OurAirports['home_link'] = np.where(df_OurAirports['home_link'].isnull(), df_OurAirports['wikipedia_link'], df_OurAirports['home_link'])

df_OurAirports.drop(['keywords','local_code','wikipedia_link','id'],axis=1, inplace=True)    # drop extra columns
our_cols = list(df_OurAirports)
new_cols = ['name','ident',  'type',  'scheduled_service', 'iso_country','continent', 
 'iso_region', 'municipality', 'latitude_deg',  'longitude_deg', 'elevation_ft', 'home_link']
df_OurAirports = df_OurAirports[new_cols]
df_OurAirports = df_OurAirports.rename(columns={'ident':'ICAO'})
df_OurAirports.sample(3)

,name,ICAO,type,scheduled_service,iso_country,continent,iso_region,municipality,latitude_deg,longitude_deg,elevation_ft,home_link
41369,Catolé da Rocha Airport,SIBU,small_airport,no,BR,SA,BR-PB,Catolé Da Rocha,-6.362530,-37.756199,892.0,NaN
23331,Aideralma Airstrip,ID-0171,small_airport,no,ID,AS,ID-PA,Aideralma,-4.334100,137.831600,2730.0,NaN
1200,Seven Feathers Airport,10FD,small_airport,no,US,NaN,US-FL,Dunnellon,29.006599,-82.404602,68.0,NaN


### <font color='black'>Step 1.1: Combine OurAirports data with Country Code location information </font>
This step ensures that each airport is assigned the correct country code for futher mapping and analysis. 

In [3]:
# Convert ISO2 to ISO3 country codes
iso_cols= ['name','alpha-2','alpha-3','region','sub-region','iso_3166-2']
df_isos = pd.read_table("data/codes.txt", sep=',',usecols=iso_cols)
#df_isos.head()

#Add country information codes to the OurAirports database
df_OurAirports_wlocations = pd.merge(df_OurAirports, df_isos, left_on='iso_country',
                                     right_on='alpha-2', how='left', sort=False)
#Now we know which airports are in which countries
df_OurAirports_wlocations = df_OurAirports_wlocations[['name_x','ICAO', 'type',
                                                       'name_y','alpha-2','alpha-3',
                                                        'scheduled_service','iso_3166-2','iso_region','region','municipality',
                                                        'latitude_deg','longitude_deg','elevation_ft','home_link']]
#rename remaining columns
df_OurAirports_wlocations.rename(columns={'name_y': 'Country_Name', 'name_x': 'Airport_Name'}, inplace=True)
#df_OurAirports_wlocations.sample(3)

# Table 2: International Civil Aviation Organization data
**Source**: ICAO has developed sophisticated tools to collect and analyze a vast array of aviation safety data metrics which allows you to identify existing and emerging global risks. ICAO along with its 192 Member States, works with Industry groups to reach consensus on international Civil Aviation Standards and Recommended Practices (SARPs) and policies.

**Link**: https://www.icao.int/safety/iStars/Pages/API-Data-Service.aspx

**Summary**: Subsets of multiple ICAO datasets are combined in the following section. The columns of interest include the following

| Field              | Description                                                              |
|--------------------|--------------------------------------------------------------------------|
|countryName		 |Name of the Country                                                       |
|countryCode		 |ISO 3-Letter Code of the Country                                          |
|airportName		 |Name of the airport, searchable                                           |
|cityName		     |Name of the city, searchable                                              |
|airportCode		 |ICAO 4-letter code of the airport                                         |
|hasInstrumentVG	 |Airport offers vertical guidance to at least one instrument runway ends   |
|elevation	         |Elevation in meters of the provided location                              |
|TerrainAbove300m	 |% of terrain raising more than 300 meters above the airport level         |
|TerrainAbove600m	 |% of terrain raising more than 600 meters above the airport level         |
|TerrainAbove900m	 |% of terrain raising more than 600 meters above the airport level         |
|hasIntersectingRWYs |Whether the airport has intersecting runways                              |
|IsInternational	 |Whether the airport is an international aerodrome                         |


### Step 2.1. Combine relevant ICAO data to main database

In [7]:
# 1. List of all international airports from ICAO
df_intl_aero = pd.read_json('data/iCAO/international_aerodromes.json') # There are 1485 international airports

# 2) Add airport safety characteristics subset to international
df_safety_data = pd.read_json('data/iCAO/international_airport_safety_characteristics.json')

# Get a useful subset of safety data df
airport_safety_data = df_safety_data[['airportCode', 'elevation', 'airnavigation_margin', 'hasIntersectingRWYs','TerrainAbove300m','TerrainAbove600m','TerrainAbove900m', 'hasInstrumentVG']]

# 3) Add airport PBN subset
df_airport_pbn = pd.read_json('data/iCAO/airport_pbn_implementation.json')

# Get a useful subset of pbn data df
df_pbn_subset = df_airport_pbn[['IsInternational','airportCode','nb_instr_vg_runways','nb_instr_runways']]

# Merge 1 and 2
df_icao1 = pd.merge(df_intl_aero, airport_safety_data, left_on='airportCode',
                                     right_on='airportCode', how='inner', sort=False)
# Merge 3
df_icao = pd.merge(df_icao1, df_pbn_subset, left_on='airportCode',
                                     right_on='airportCode', how='inner', sort=False)

# Combine with the rest of the database
df_OurAirports_wlocations_ICAO1 = pd.merge(df_OurAirports_wlocations, df_icao, left_on='ICAO',
                                     right_on='airportCode', how='right', sort=False)

# Resulting dataframe contains all information on about 1477 airports (all international)
df_OurAirports_wlocations_ICAO1.sample(3)

,Airport_Name,ICAO,type,Country_Name,alpha-2,alpha-3,scheduled_service,iso_3166-2,iso_region,region,municipality,latitude_deg,longitude_deg,elevation_ft,home_link,airportCode,airportName,cityName,countryCode,countryName,geometry,latitude,longitude,elevation,airnavigation_margin,hasIntersectingRWYs,TerrainAbove300m,TerrainAbove600m,TerrainAbove900m,hasInstrumentVG,IsInternational,nb_instr_vg_runways,nb_instr_runways
798,Arad International Airport,LRAR,medium_airport,Romania,RO,ROU,yes,ISO 3166-2:RO,RO-AR,Europe,Arad,46.176601,21.261999,352.0,http://en.wikipedia.org/wiki/Arad_Internationa...,LRAR,Arad,Arad,ROU,Romania,"{'type': 'Point', 'coordinates': [21.261944444...",46.176667,21.261944,103.0,26.68,False,0.0,0.00,0.0,1.0,True,1,2
882,Sangster International Airport,MKJS,medium_airport,Jamaica,JM,JAM,yes,ISO 3166-2:JM,JM-08,Americas,Montego Bay,18.503700,-77.913399,4.0,http://en.wikipedia.org/wiki/Sangster_Internat...,MKJS,Sangster Intl,Montego Bay,JAM,Jamaica,"{'type': 'Point', 'coordinates': [-77.91333333...",18.503611,-77.913333,14.0,18.33,False,10.5,0.00,0.0,1.0,True,1,2
616,Split Airport,LDSP,medium_airport,Croatia,HR,HRV,yes,ISO 3166-2:HR,HR-17,Europe,Split,43.538898,16.298000,79.0,http://en.wikipedia.org/wiki/Split_Airport,LDSP,Kastela,Split,HRV,Croatia,"{'type': 'Point', 'coordinates': [16.297963888...",43.538944,16.297964,16.0,4.90,False,33.0,0.63,0.0,1.0,True,1,2


# <font color='black'> Table 3: INFORM 2017 </font>
**Source**:
The file "data/INFORM_2017_v031.xlsx" includes country-level risk information. 
        
**Link**: http://www.inform-index.org/

**Summary**: The INFORM initiative began in 2012 as a convergence of interests of UN agencies, donors, NGOs and research institutions to establish a common evidence-base for global humanitarian risk analysis. INFORM identifies the countries at a high risk of humanitarian crisis that are more likely to require international assistance. The INFORM model is based on risk concepts published in scientific literature and envisages three dimensions of risk: Hazards & Exposure, Vulnerability and Lack of Coping Capacity. The INFORM model is split into different levels to provide a quick overview of the underlying factors leading to humanitarian risk. The INFORM index supports a proactive crisis management framework. It will be helpful for an objective allocation of resources for disaster management as well as for coordinated actions focused on anticipating, mitigating, and preparing for humanitarian emergencies.



In [8]:
df_inform_main = pd.read_excel('data/INFORM_2017_v031.xlsx', sheetname='INFORM 2017 (a-z)', skiprows=0)

df_inform_main.drop(['Reliability Index (*)','Number of Missing Indicators', '% of Missing Indicators',
                    'Countries in HVC','Recentness data (average years)','Human','Development & Deprivation', 
                     'Inequality','Socio-Economic Vulnerability', 'Recent Shocks','DRR','Communication',
                    'Access to health care','Uprooted people','Health Conditions','Children U5','Food Security',
                     'Other Vulnerable Groups','Vulnerable Groups','Governance','Institutional','Rank','Infrastructure'], axis=1, inplace=True)

df_inform_main.rename(columns={'Earthquake':'Earthquake (0-10)','Flood':'Flood (0-10)',
                               'Tsunami':'Tsunami (0-10)','Tropical Cyclone':'Tropical Cyclone (0-10)',
                               'Drought':'Drought (0-10)','Natural':'Natural (0-10)',
                               'Projected Conflict Risk':'Projected Conflict Risk (0-10)',
                               'Current Highly Violent Conflict Intensity':'Current Highly Violent Conflict Intensity (0-10)',
                               'HAZARD & EXPOSURE' : 'HAZARD & EXPOSURE (1-10)',
                               'Aid Dependency':'Aid Dependency (1-10)',
                               'VULNERABILITY':'VULNERABILITY (1-10)',
                               'Physical infrastructure':'Physical infrastructure (1-10)',
                               'LACK OF COPING CAPACITY':'LACK OF COPING CAPACITY (0-10)',
                               'INFORM RISK':'INFORM RISK (0-10)', 'RISK CLASS':'RISK CLASS (Very Low-Very High)', 
                                }, inplace=True)

df_inform_main.drop(df_inform_main.index[0],inplace=True)
df_inform_main.sample(3)

,COUNTRY,ISO3,Earthquake (0-10),Flood (0-10),Tsunami (0-10),Tropical Cyclone (0-10),Drought (0-10),Natural (0-10),Projected Conflict Risk (0-10),Current Highly Violent Conflict Intensity (0-10),HAZARD & EXPOSURE (1-10),Aid Dependency (1-10),VULNERABILITY (1-10),Physical infrastructure (1-10),LACK OF COPING CAPACITY (0-10),INFORM RISK (0-10),RISK CLASS (Very Low-Very High)
40,Costa Rica,CRI,9.6,3.5,8.7,2,0.8,6.3,0.1,0,3.8,0.2,2.3,2.2,2.8,2.9,Low
142,Saint Kitts and Nevis,KNA,0.1,0.1,0,6.3,0,1.7,0,0,0.9,10,3.3,0.6,3.5,2.2,Low
78,Iraq,IRQ,7,9.6,0,0,3.9,5.5,10,9,7.7,1.9,6,4.4,7,6.9,Very High


### <font color='black'>Step 3.1: Add Country-Level INFORM risk data to each row </font>
Most global risk assessment studies have been performed at a country level. As a starting point, each airport is linked with its country-level vulnerability ranking (given in the INFORM database). Afterwards, finer detail is incorporated to analyze vulnerability at the airport-level.  

In [9]:
#Add different country information codes to the OurAirports database
df_step3 = pd.merge(df_OurAirports_wlocations_ICAO1, df_inform_main, left_on='countryCode',
                                     right_on='ISO3', how='left', sort=False)

#Drop duplicate columns
df_step3.drop(['COUNTRY','ISO3'],axis=1, inplace=True)

#Now we know the country-level risk of each airport
df_step3.sample(3)

,Airport_Name,ICAO,type,Country_Name,alpha-2,alpha-3,scheduled_service,iso_3166-2,iso_region,region,municipality,latitude_deg,longitude_deg,elevation_ft,home_link,airportCode,airportName,cityName,countryCode,countryName,geometry,latitude,longitude,elevation,airnavigation_margin,hasIntersectingRWYs,TerrainAbove300m,TerrainAbove600m,TerrainAbove900m,hasInstrumentVG,IsInternational,nb_instr_vg_runways,nb_instr_runways,Earthquake (0-10),Flood (0-10),Tsunami (0-10),Tropical Cyclone (0-10),Drought (0-10),Natural (0-10),Projected Conflict Risk (0-10),Current Highly Violent Conflict Intensity (0-10),HAZARD & EXPOSURE (1-10),Aid Dependency (1-10),VULNERABILITY (1-10),Physical infrastructure (1-10),LACK OF COPING CAPACITY (0-10),INFORM RISK (0-10),RISK CLASS (Very Low-Very High)
773,Ovda International Airport,LLOV,large_airport,Israel,IL,ISR,yes,ISO 3166-2:IL,IL-D,Asia,Eilat,29.940300,34.935799,1492.0,http://en.wikipedia.org/wiki/Ovda_Internationa...,LLOV,Ovda,Ovda,ISR,Israel,"{'type': 'Point', 'coordinates': [34.940833333...",29.935000,34.940833,438.0,18.32,False,2.25,0.00,0.00,1.0,True,1,1,6.6,2.4,5.2,0,5.3,4.3,6.4,0,4.4,0,2.1,0,2.3,2.8,Low
1399,Kota Kinabalu International Airport,WBKK,medium_airport,Malaysia,MY,MYS,yes,ISO 3166-2:MY,MY-12,Asia,Kota Kinabalu,5.937210,116.051003,10.0,http://en.wikipedia.org/wiki/Kota_Kinabalu_Int...,WBKK,Kota Kinabalu,Kota Kinabalu,MYS,Malaysia,"{'type': 'Point', 'coordinates': [116.05305555...",5.944722,116.053056,33.0,-17.88,False,6.44,1.88,0.88,1.0,True,2,2,4.1,6.5,6.2,2.8,3.4,4.8,5.1,0,4.2,0,3,2.9,3.1,3.4,Low
100,Munich Airport,EDDM,large_airport,Germany,DE,DEU,yes,ISO 3166-2:DE,DE-BY,Europe,Munich,48.353802,11.786100,1487.0,http://www.munich-airport.com/,EDDM,Munich,Munich,DEU,Germany,"{'type': 'Point', 'coordinates': [11.786086111...",48.353783,11.786086,445.0,-1.52,False,0.00,0.00,0.00,1.0,True,4,4,2.7,6.1,0,0,0.5,2.2,0.1,0,1.2,0,2.4,0,1.5,1.6,Very Low


# <font color='black'> Table 4: HDX World Food Program Airport Data </font>
**Source**: The files associated with "wld_trs_airports_wfp" are available on the Humanitarian Data Exchange website.

**Link**: https://data.humdata.org/dataset/global-logistics

**Summary**: This dataset brings together various public sources such as OpenStreetMap or ourairports.com with WFP logistics information. It is updated regularly with inputs from WFP aviation unit but also from many partners through the Logistics Cluster and the Logistics Capacity Assessment (LCA: dlca.logcluster.org). The information is compiled at a global level by the Emergency and Preparedness Geospatial Information Unit at the World Food Programme (WFP) Headquarters in Rome, Italy.This dataset is at a global scale and is updated country by country. The last update date can be retrieved from the data of the country of interest.


### Step 4.1: Add Airport-specific humanitarian data
Until this step, all airports within a country have been assigned the same country-level vulnerability information. This step adds airport-level information from the World Food Program. 

In [10]:
wfp_airports = gpd.read_file("data/HDX_data/wld_trs_airports_wfp")  #WFP includes information on 3513 airports
wfp_airports.dropna(subset=['icao'], inplace=True)
#wfp_airports.head(3)

# Add WFP humanitarian airport information to the main database
df_step4 = pd.merge(df_step3, wfp_airports, left_on='ICAO',
                                     right_on='icao', how='left', sort=False)

# Manage columns
#wfp_cols = list(df_step4)
new_wfp_cols = [ 'airportCode', 'airportName', 'cityName', 'countryCode', 'countryName','latitude_x', 'longitude_x',
 #'geometry_x',
  'region', 'municipality', 'type', 'scheduled_service', 'home_link','elevation',
  'TerrainAbove300m', 'TerrainAbove600m', 'TerrainAbove900m','airnavigation_margin', 'hasIntersectingRWYs',
 'hasInstrumentVG', 'IsInternational', 'nb_instr_vg_runways', 'nb_instr_runways',
 'Earthquake (0-10)', 'Flood (0-10)', 'Tsunami (0-10)', 'Tropical Cyclone (0-10)', 'Drought (0-10)',
 'Natural (0-10)', 'Projected Conflict Risk (0-10)', 'Current Highly Violent Conflict Intensity (0-10)',
 'HAZARD & EXPOSURE (1-10)', 'Aid Dependency (1-10)', 'VULNERABILITY (1-10)', 'Physical infrastructure (1-10)',
 'LACK OF COPING CAPACITY (0-10)', 'INFORM RISK (0-10)', 'RISK CLASS (Very Low-Very High)',
  'apttype', 'aptclass', 'status', 'dmg', 'rwpaved', 'rwlengthm',  'elevm',  'humuse', 'humoperate',
 ]

df_step4 = df_step4[new_wfp_cols]

df_step4.sample(3)

,airportCode,airportName,cityName,countryCode,countryName,latitude_x,longitude_x,region,municipality,type,scheduled_service,home_link,elevation,TerrainAbove300m,TerrainAbove600m,TerrainAbove900m,airnavigation_margin,hasIntersectingRWYs,hasInstrumentVG,IsInternational,nb_instr_vg_runways,nb_instr_runways,Earthquake (0-10),Flood (0-10),Tsunami (0-10),Tropical Cyclone (0-10),Drought (0-10),Natural (0-10),Projected Conflict Risk (0-10),Current Highly Violent Conflict Intensity (0-10),HAZARD & EXPOSURE (1-10),Aid Dependency (1-10),VULNERABILITY (1-10),Physical infrastructure (1-10),LACK OF COPING CAPACITY (0-10),INFORM RISK (0-10),RISK CLASS (Very Low-Very High),apttype,aptclass,status,dmg,rwpaved,rwlengthm,elevm,humuse,humoperate
553,KPDX,Portland Intl,Portland,USA,USA,45.588708,-122.596867,Americas,Portland,large_airport,yes,http://en.wikipedia.org/wiki/Portland_Internat...,15.0,0.63,0.00,0.00,-7.05,False,1.0,True,5,5,7.9,6.3,7.3,7.6,4.7,6.9,6.2,0,5.8,0,2.3,1,2.2,3.1,Low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
70,DRZR,Zinder,Zinder,NER,Niger,13.783972,8.990403,Africa,Zinder,medium_airport,yes,http://en.wikipedia.org/wiki/Zinder_Airport,441.0,0.00,0.00,0.00,23.25,False,0.0,True,0,2,0.1,7.1,0,0,8.3,4.2,8.1,9,7.3,4.7,7,9.3,7.7,7.3,Very High,Airport,International,Open,Unknown,Yes,1825.0,462.0,Connection,UNHAS
620,LEAM,Almeria,Almeria,ESP,Spain,36.843889,-2.370000,Europe,Almería,medium_airport,yes,http://www.aena.es/csee/Satellite?cid=10481468...,30.0,22.81,11.44,5.44,-2.58,False,1.0,True,2,2,4.3,5.5,6.3,0,5.1,4.5,4,0,3.7,0,1.3,0,1.9,2.1,Low,Unknown,Unknown,Open,Unknown,Unknown,NaN,15.0,No,None


## Step 5: Add total number of airports in each country (from CIA Factbook)
An airport is more critical for humanitarian operations if it is the only international airport that serves that community, as was the case of the Kathmandu airport in the 2015 Nepal earthquake. 

In [11]:
# Get a list of how many airports are in each country from CIA website
df_count = pd.read_csv('data/airports_per_country_count_CIA.csv', header =0, index_col=0)

# Clean up country names in the CIA file. Get UNSD list of country names 
names = ['ISO3','Country Name']
df_UNSD_countries = pd.read_excel('data/UNSD_countries.xlsx', header=None, names=names)

# CIA country names are messy. Combine with CIA iso3 list. Then merge number of airports per country to main database.
cia_names=['Country', 'GEC','iso2','iso3','idk','NATO','internet','comment']
df_CIA_countries = pd.read_excel('data/CIA_countries.xlsx',names=cia_names)
df_CIA_countries=df_CIA_countries[['Country','iso3']]
df_CIA_countries.count()

## Verification Script ## To test against an official list
unverified_list = df_isos['alpha-3']
verified_list = df_UNSD_countries['ISO3']
# how many match the UNSD?
#df_matching = unverified_list[unverified_list.isin(verified_list)]
#df_matching.nunique()

replace_country_values = {'United States of America':'United States', 
                          'Russian Federation':'Russia',
                          'Bolivia (Plurinational State of)   ':'Bolivia',
                          'United Kingdom of Great Britain and Northern Ireland ':'United Kingdom',
                          'Venezuela (Bolivarian Republic of) ':'Venezuela', 
                          'Iran (Islamic Republic of)':'Iran',
                          'Congo ':'Congo (Democratic Republic of the) ',
                          'Tanzania, United Republic of ':'Tanzania', 
                          'Syrian Arab Republic':'Syria', 
                          'Bahamas':'Bahamas, The',
                          'Viet Nam':'Vietnam', 
                          "Lao People's Democratic Republic  ":'Laos',
                          'Taiwan, Province of China':'Taiwan',
                          'Swaziland':'Eswatini', 
                          'Gambia':'Gambia, The',
                          'Sint Maarten (Dutch part)  ':'Saint Martin'
                         }                                                                                          

df_count_cleaner = df_count.replace({"Country": replace_country_values})  
#df_count_cleaner.info() #241 Countries with airport information

CIA_airports_count = pd.merge(df_CIA_countries, df_count_cleaner, left_on='Country',
                                     right_on='Country', how='left', sort=False)
#Now the CIA airports count has ISO3 code. Use code to combine with other data. 

df_step5 = pd.merge(df_step4, CIA_airports_count, left_on='countryCode',
                                     right_on='iso3', how='left', sort=False)
df_step5.sample(3)

,airportCode,airportName,cityName,countryCode,countryName,latitude_x,longitude_x,region,municipality,type,scheduled_service,home_link,elevation,TerrainAbove300m,TerrainAbove600m,TerrainAbove900m,airnavigation_margin,hasIntersectingRWYs,hasInstrumentVG,IsInternational,nb_instr_vg_runways,nb_instr_runways,Earthquake (0-10),Flood (0-10),Tsunami (0-10),Tropical Cyclone (0-10),Drought (0-10),Natural (0-10),Projected Conflict Risk (0-10),Current Highly Violent Conflict Intensity (0-10),HAZARD & EXPOSURE (1-10),Aid Dependency (1-10),VULNERABILITY (1-10),Physical infrastructure (1-10),LACK OF COPING CAPACITY (0-10),INFORM RISK (0-10),RISK CLASS (Very Low-Very High),apttype,aptclass,status,dmg,rwpaved,rwlengthm,elevm,humuse,humoperate,Country,iso3,Count
200,EHGG,Eelde,Groningen,NLD,Netherlands,53.125000,6.583333,Europe,Groningen,medium_airport,yes,http://en.wikipedia.org/wiki/Groningen_Airport...,3.0,0.00,0.00,0.00,6.78,True,1.0,True,1,2,1.7,5.8,0,0,0.5,1.9,0,0,1,0,2.1,0.1,1.2,1.4,Very Low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Netherlands,NLD,29.0
631,LELL,Sabadell,Sabadell,ESP,Spain,41.520833,2.105000,Europe,Sabadell,small_airport,no,NaN,142.0,17.19,2.31,0.06,-2.58,False,0.0,True,0,0,4.3,5.5,6.3,0,5.1,4.5,4,0,3.7,0,1.3,0,1.9,2.1,Low,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Spain,ESP,150.0
300,FCPP,Antonio Agostinho Neto,Pointe Noire,COG,"Congo, Rep Of",-4.814806,11.886278,Africa,Pointe Noire,medium_airport,yes,http://en.wikipedia.org/wiki/Pointe_Noire_Airport,8.0,0.00,0.00,0.00,-18.61,False,1.0,True,1,2,1.6,7.2,0,0,0.5,2.5,7.8,0,4.2,0.8,4.8,8,7.3,5.3,High,Airport,International,Open,Unknown,Yes,2600.0,17.0,No,None,"Congo, Republic of the",COG,27.0


## Table 6: Create an "Isolated Countries List" to evaluate criticality
### Step 6.2 determine the country's degree of isolation
If a nation only has one or very few international airports in the entire country, that does not necessarily automatically raise the criticality of those airports. Luxembourg, for instance, has few airports but is such a small country with good road and rail networks that overland transport would be a feasible alternative. "Isolated" nations therefore, are ones with challenging mountaneous terrain, political turmoil, or are island nations.

**Load 'Isolated Countries' List**
* Islands were added if they are a part of the Small Island Developing States list according to the UN: https://sustainabledevelopment.un.org/topics/sids/list



In [13]:
# NOTE: CIA does not have data for some countries. 
# These are very small (island) countries and will be added to the 'Isolated' countries list
#test = df_OurAirports_wlocations_risk_andcount[df_OurAirports_wlocations_risk_andcount['Count'].isnull()]
#test.Country_Name.value_counts()

df_isolated = pd.read_excel('data/Isolated_countries_list.xlsx')
#df_isolated.head()
iso_list_1 = df_isolated[df_isolated['Isolated'] == 1]
iso_list = iso_list_1['ISO3'].values.tolist() #List of isolated countries
print(iso_list)

['ABW', 'AFG', 'AIA', 'AND', 'ASM', 'ATA', 'ATG', 'BDI', 'BHS', 'BLZ', 'BMU', 'BRB', 'BTN', 'CHL', 'COD', 'COG', 'COK', 'COM', 'CPV', 'CUB', 'CYM', 'DMA', 'DOM', 'FJI', 'FSM', 'GLP', 'GNB', 'GRD', 'GUF', 'GUM', 'GUY', 'HTI', 'JAM', 'KGZ', 'KIR', 'KNA', 'LCA', 'LSO', 'MAF', 'MCO', 'MDG', 'MDV', 'MHL', 'MMR', 'MNP', 'MSR', 'MTQ', 'MUS', 'MYT', 'NCL', 'NER', 'NIU', 'NPL', 'NRU', 'PLW', 'PRI', 'PYF', 'REU', 'SGP', 'SLB', 'SOM', 'STP', 'SUR', 'SYC', 'TCA', 'TJK', 'TLS', 'TON', 'TTO', 'TUV', 'UMI', 'VCT', 'VGB', 'VIR', 'VUT', 'WSM']


### <font color='black'> Step 6.3: Procedurally/Politically Isolated  </font>
**Source**: World Bank Data Catalog - World Bank: Time to import. World Bank, Doing Business project (doingbusiness.org).

**Link**: https://data.worldbank.org/indicator/IC.IMP.DURS?view=chart

**Summary**: Gives the number of days necessary to comply with all procedures required to import goods. The waiting time between procedures – for example, during unloading of the cargo – is included in the measure. Assumption is that country’s with high time to import would face even longer delays in a disaster scenario. 


In [295]:
df_wb_tti = pd.read_excel('data/WorldBank_TimeToImport.xls', sheetname='Data',header=3)

df_wb_tti= df_wb_tti.drop(df_wb_tti.iloc[:, 3:58], axis=1)
df_wb_tti.drop(['2017', '2016','2015','Indicator Name'], axis=1, inplace=True)
df_wb_tti.rename(columns={'2014':'World Bank Time to import (days) 2014'}, inplace=True)
df_wb_tti['World Bank Time to import (days) 2014'] = df_wb_tti['World Bank Time to import (days) 2014'].astype(object)

procedurally_isolated = df_wb_tti[df_wb_tti['World Bank Time to import (days) 2014']>50]
procedurally_isolated_list = procedurally_isolated['Country Code'].values.tolist()

# Step 7: Assign Criticality points
| Parameter                            | Point assignment                                      |
|--------------------------------------|-------------------------------------------------------|
| Aerodrome Type                       | "Airport" = 1, "heliport, airfield, or airstrip" = 0  |
| Aerodrome Class                      | "International" = 3, "Domestic" = 0                   |
| Aerodrome Size                       | "Large" = 3, "Medium" = 1, "Small" = 0                |
| Runway Paved?                        | "Yes" = 0, "No" = -1                                  |
| Island Nation?                       | "Yes" = 5, "No" = 0                                   |
| Number Airports in Country           | "Less than three" = 5, "More than three" = 0          |
| Dangerous Terrain                    | ">900m" = 3, ">600m" = 2, ">300m" = 1                 |
| Procedurally Isolated                | "Time to Import Cargo >50 days" = 2                   |
| Number Airports in Country           | "Less than three" = 5, "More than three" = 0          |
| TOTAL Criticality Points             | Sum of parameters                                     |

In [297]:
df_final = df_step5.copy()
df_final['crit_airfield'] = np.where(df_final['apttype']=='Airport', 1, 0)
#df_final['crit_international'] = np.where(df_final['IsInternational']==True, 3, 0) NOTE: ALL INTERNATIONAL 
df_final['crit_size'] = np.where(df_final['type']=='large_airport', 3, 0)
df_final['crit_size'] = np.where(df_final['type']=='medium_airport', 1, 0)
df_final["crit_Island"] = np.where(df_final["countryCode"].isin(iso_list), 5, 0)
df_final['crit_und3airpts']  = np.where(df_final['Count'] <= 3, 5, 0)   # should be dependent on airport size too
df_final['crit_terr300'] = np.where(df_final['TerrainAbove300m'] != 0, 1, 0)
df_final['crit_terr600'] = np.where(df_final['TerrainAbove600m'] != 0, 1, 0)
df_final['crit_terr900'] = np.where(df_final['TerrainAbove900m'] != 0, 1, 0)
df_final["crit_Isolated"] = np.where(df_final["countryCode"].isin(procedurally_isolated_list), 2, 0)
df_final['crit_metric']= df_final["crit_Isolated"]+df_final['crit_airfield'] + df_final['crit_size']+ df_final["crit_Island"] + df_final['crit_und3airpts']+df_final['crit_terr300']+df_final['crit_terr600']+df_final['crit_terr900']
df_final['crit_metric']  = np.where(df_final['crit_metric'] <0, 0, df_final['crit_metric']) #No negative values


# Step 8: Vulnerability points
| Parameter                            | Point assignment                                      |
|--------------------------------------|-------------------------------------------------------|
| Country-level Vulnerability          | "Most Vulnerable" = 10, "Least Vulnerable" = 0        |
| Country-level Hazard & Exposure      | "Most Vulnerable" = 10, "Least Vulnerable" = 0        |
| Country-level Lack of Coping Capcity | "Most Vulnerable" = 10, "Least Vulnerable" = 0        |
| Elevation of Airport                 | "At risk" = 5                                         |
| Size of Airport                      | "Large" = 0, "Medium" = 1, "Small" = 3                |
| Runway Vertical Guidance Instruments | "No runways have VG" = 2                              |
| Intersecting Runways                 | "Airport has intersecting runways" = 2                |
| Runways paved?                       | "Airport runways are unpaved" = 3                     |
| Runways damaged?                     | "WFP reported damaged runways" = 3                    |
| TOTAL Vulnerability Points             | Sum of parameters                                     |

In [299]:
df_final['vuln_flood_elev'] = np.where(df_final['elevation']<10 , df_final['Flood (0-10)']*2, 0)
df_final['vuln_tsunami_elev'] = np.where(df_final['elevation']<10  , df_final['Tsunami (0-10)']*2, 0)
df_final['vuln_cyclone_elev'] = np.where(df_final['elevation']<10 , df_final['Tropical Cyclone (0-10)']*2, 0)
df_final['vuln_elev_actual'] = df_final['vuln_flood_elev'] +df_final['vuln_tsunami_elev']+df_final['vuln_cyclone_elev']
df_final['vuln_elev'] = (df_final['vuln_elev_actual'].astype(float).round(decimals=-1))/10
df_final['vuln_size'] = np.where(df_final['type']=='small_airport', 3, 0)
df_final['vuln_size'] = np.where(df_final['type']=='medium_airport', 1, 0)
df_final['vuln_InstruVG'] = np.where(df_final['hasInstrumentVG'] != 0, 2, 0)
df_final['vuln_Intersect'] = np.where(df_final['hasIntersectingRWYs'] != 0, 2, 0)
df_final['vuln_paved'] = np.where(df_final['rwpaved']=='No', 3, 0)
df_final['vuln_dmg'] = np.where(df_final['dmg']=='Yes', 3, 0)

df_final['vuln_inform'] = ((df_final['HAZARD & EXPOSURE (1-10)']+df_final['VULNERABILITY (1-10)']+df_final['LACK OF COPING CAPACITY (0-10)'])/3)
#.astype(float).round(decimals=0)
df_final['vuln_metric'] =(df_final['vuln_inform']+(df_final['vuln_elev']+df_final['vuln_size']+df_final['vuln_InstruVG']+df_final['vuln_Intersect']+df_final['vuln_paved']+df_final['vuln_dmg'])/2).astype(float).round(decimals=0)

In [307]:
df_final['crit_metric']  = np.where(df_final['crit_metric'] >10, 10, df_final['crit_metric']) #No values greater than 10
df_final['vuln_metric']  = np.where(df_final['vuln_metric'] >10, 10, df_final['vuln_metric']) #No values greater than 10
df_final['vuln_metric']  = df_final['vuln_metric'].fillna(5)

In [308]:
df_final2=df_final.drop(['crit_airfield','crit_size','crit_Island','crit_und3airpts','crit_terr300','crit_terr600','crit_terr900',
                                  'crit_Isolated','vuln_flood_elev','vuln_tsunami_elev','vuln_cyclone_elev','vuln_elev_actual','vuln_size',
                                  'vuln_InstruVG','vuln_Intersect','vuln_paved','vuln_dmg','vuln_inform','vuln_elev'], axis=1)

### Export to Excel

In [311]:
writer = pd.ExcelWriter('draft_output_v8.xlsx')
df_final2.to_excel(writer,'All Data')
#df_inform_meta.to_excel(writer,'Terms Used')
writer.save()

In [ ]:
#random


In [92]:
testing = pd.read_csv('data/worldcitiespop/worldcitiespop.txt', encoding = "latin1", sep=',')

C:\Users\shann\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [91]:
testing.sample(5)

,Country,City,AccentCity,Region,Population,Latitude,Longitude
132907,ar,morteros,Morteros,05,NaN,-30.711640,-61.998624
1472885,ir,qiun qeshlaqi,Qiun Qeshlaqi,33,NaN,37.384500,46.992800
1719615,lt,skirsnemunes,Skirsnemunes,62,NaN,55.095000,22.906111
2339992,ro,toporasti,Toporasti,38,NaN,46.700000,27.416667
940892,fr,la berliere,La Berlière,B4,NaN,50.302061,3.882094


In [ ]:

The main 'geoname' table has the following fields :
---------------------------------------------------
geonameid         : integer id of record in geonames database
name              : name of geographical point (utf8) varchar(200)
asciiname         : name of geographical point in plain ascii characters, varchar(200)
alternatenames    : alternatenames, comma separated, ascii names automatically transliterated, convenience attribute from alternatename table, varchar(10000)
latitude          : latitude in decimal degrees (wgs84)
longitude         : longitude in decimal degrees (wgs84)
feature class     : see http://www.geonames.org/export/codes.html, char(1)
feature code      : see http://www.geonames.org/export/codes.html, varchar(10)
country code      : ISO-3166 2-letter country code, 2 characters
cc2               : alternate country codes, comma separated, ISO-3166 2-letter country code, 200 characters
admin1 code       : fipscode (subject to change to iso code), see exceptions below, see file admin1Codes.txt for display names of this code; varchar(20)
admin2 code       : code for the second administrative division, a county in the US, see file admin2Codes.txt; varchar(80) 
admin3 code       : code for third level administrative division, varchar(20)
admin4 code       : code for fourth level administrative division, varchar(20)
population        : bigint (8 byte int) 
elevation         : in meters, integer
dem               : digital elevation model, srtm3 or gtopo30, average elevation of 3''x3'' (ca 90mx90m) or 30''x30'' (ca 900mx900m) area in meters, integer. srtm processed by cgiar/ciat.
timezone          : the iana timezone id (see file timeZone.txt) varchar(40)
modification date : date of last modification in yyyy-MM-dd format

In [99]:
allcountries = pd.read_table('data/worldcitiespop/allCountries.txt', 
                             header=None, names=['geonameid','name','asciiname','alternatenames','latitude','longitude',
                            'feature class','feature code','country code','cc2','admin1 code','admin2 code', 'admin3 code',
                             'admin4 code','population','elevation','dem','timezone', 'modification date'] )

C:\Users\shann\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (9,10,11,12,13) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [114]:
countries_subset = allcountries[['asciiname','latitude','longitude','country code','admin1 code','admin2 code',
                                 'admin3 code','admin4 code','population','elevation']]
#countries_subset.info(verbose=True,null_counts=True)

In [113]:
countries_subset_nonzero= countries_subset[countries_subset.population != 0]
countries_subset_nonzero.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 516485 entries, 192 to 11784656
Data columns (total 10 columns):
asciiname       516461 non-null object
latitude        516485 non-null float64
longitude       516485 non-null float64
country code    516362 non-null object
admin1 code     516220 non-null object
admin2 code     478368 non-null object
admin3 code     266184 non-null object
admin4 code     99835 non-null object
population      516485 non-null int64
elevation       71889 non-null float64
dtypes: float64(3), int64(1), object(6)
memory usage: 43.3+ MB
